# Problem VII: Analyzing Policing Data from Chicago and California

**Version 0.1**

In this notebook you are asked to identify correlations in a data set that includes information about crimes that have been collected both in the city of Chicago and in California. You will find correlations between different variables and be asked to supply interpretations for the correlations that have been identified. 

* * *

By Tiffany Nichols (Northeastern University)  
24 October 2024

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib ipympl
# %matplotlib inline
# %matplotlib notebook

from scipy.stats.contingency import association
import seaborn as sns
from scipy.stats import chi2_contingency
import plotly.express as px


## Problem 1) Chicago PD Data

**Problem 1a** 

Load the data from Chicago. What columns (i.e., variables) are present within the data? 

In [13]:
chicago_df = pd.read_csv("ChicagoPolicingData.csv")
chicago_df

,TIME_PERIOD,TIME_PERIOD_START,TIME_PERIOD_END,PRIMARY_TYPE,AGE,SEX,RACE,JUVENILE_I,DOMESTIC_I,GUNSHOT_INJURY_I,NUMBER_OF_VICTIMS
0,2011 Q1,01/01/2011,03/31/2011,HOMICIDE,30-39,M,BLK,False,False,NO,1
1,2000 Q4,10/01/2000,12/31/2000,HOMICIDE,70-79,M,WHI,False,False,NO,2
2,1992 Q4,10/01/1992,12/31/1992,HOMICIDE,70-79,M,WHI,False,False,NO,1
3,2012 Q4,10/01/2012,12/31/2012,BATTERY,20-29,UNKNOWN,UNKNOWN,False,False,NO,1
4,2003 Q2,03/01/2003,06/30/2003,CRIMINAL SEXUAL ASSAULT,0-19,F,WWH,True,False,UNKNOWN,42
...,...,...,...,...,...,...,...,...,...,...,...
54856,2024 Q3,07/01/2024,09/30/2024,ROBBERY,30-39,F,WWH,False,True,NO,4
54857,2024 Q3,07/01/2024,09/30/2024,ROBBERY,0-19,M,BLK,False,False,YES,3
54858,2024 Q3,07/01/2024,09/30/2024,ASSAULT,20-29,M,WBH,False,False,NO,11
54859,2024 Q3,07/01/2024,09/30/2024,ROBBERY,30-39,F,WBH,False,False,NO,4


In [14]:
chicago_df.columns

Index(['TIME_PERIOD', 'TIME_PERIOD_START', 'TIME_PERIOD_END', 'PRIMARY_TYPE',
       'AGE', 'SEX', 'RACE', 'JUVENILE_I', 'DOMESTIC_I', 'GUNSHOT_INJURY_I',
       'NUMBER_OF_VICTIMS'],
      dtype='object')

In [15]:
chicago_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54861 entries, 0 to 54860
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   TIME_PERIOD        54861 non-null  object
 1   TIME_PERIOD_START  54861 non-null  object
 2   TIME_PERIOD_END    54861 non-null  object
 3   PRIMARY_TYPE       54861 non-null  object
 4   AGE                54861 non-null  object
 5   SEX                54861 non-null  object
 6   RACE               54861 non-null  object
 7   JUVENILE_I         51749 non-null  object
 8   DOMESTIC_I         54861 non-null  bool  
 9   GUNSHOT_INJURY_I   54861 non-null  object
 10  NUMBER_OF_VICTIMS  54861 non-null  int64 
dtypes: bool(1), int64(1), object(9)
memory usage: 4.2+ MB


*write your response here*

The variables in this data set are 'TIME_PERIOD', 'TIME_PERIOD_START', 'TIME_PERIOD_END', 'PRIMARY_TYPE', 'AGE', 'SEX', 'RACE', 'JUVENILE_I', 'DOMESTIC_I', 'GUNSHOT_INJURY_I', 'NUMBER_OF_VICTIMS'.

**Problem 1b**

Prior to any numerical analysis which variable do you expect to be most strongly correlated with `PRIMARY TYPE` (the charge at the time of arrest)?

*write your answer here*

**Problem 1c**

Using the available data identify any trends or correlations present within the data. 

Do you identify any particularly strong correlations? 

In [29]:
def cramers_v(confusion_matrix):
    chi2, p, dof, expected = chi2_contingency(confusion_matrix)
    n = confusion_matrix.sum().sum()
    #print(n)
    return np.sqrt(chi2 / (n * (min(confusion_matrix.shape) - 1)))

# List of variables for the correlation
variables = ['TIME_PERIOD', 'TIME_PERIOD_START', 'TIME_PERIOD_END', 'PRIMARY_TYPE', 'AGE', 'SEX', 'RACE', 'JUVENILE_I', 'DOMESTIC_I', 'GUNSHOT_INJURY_I', 'NUMBER_OF_VICTIMS']

# Initialize an empty DataFrame to store Cramér's V values
cramers_v_matrix = pd.DataFrame(index=variables, columns=variables)

# Calculate pairwise Cramér's V for all variables
for var1 in variables:
    for var2 in variables:
        if var1 == var2:
            cramers_v_matrix.loc[var1, var2] = 1.0  # Cramér's V of a variable with itself is 1
        else:
            confusion_matrix = pd.crosstab(chicago_df[var1], chicago_df[var2])
            cramers_v_matrix.loc[var1, var2] = cramers_v(confusion_matrix)
            

# Convert to numeric for the heatmap
cramers_v_matrix = cramers_v_matrix.astype(float)
cramers_v_matrix

# Create a heatmap
fig = px.imshow(cramers_v_matrix, text_auto=True,  width=1200, height=1200, color_continuous_scale='Viridis')
fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
fig.show()

*write your answer here*

## Problem 2) California Data

The data from California is spread across multiple files. 

**Problem 2a** 

Load the table defining the columns in the data set.

Take a moment to read the variables and their definitions.

In [30]:
dd_df = pd.read_csv("DataDictionary.csv")
dd_df

,Name,Definition,Type,Width/Format,Coding/Comments
0,ind_id,Indicator ID,N,3,752
1,ind_definition,Definition of indicator in plain language,T,255,Free text
2,reportyear,Year(s) that the indicator was reported,N,4,"2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007..."
3,race_eth_code,numeric code for a race/ethnicity group,N,1,9=Total
4,race_eth_name,Name of race/ethnic group,T,5,Total
5,geotype,Type of geographic unit,T,2,"PL: Place (including city, town and census des..."
6,geotypevalue,Value of geographic unit,N,5,"5-digit FIPS place code, 5-digit FIPS county c..."
7,geoname,Name of geographic unit,T,30,"place name, county name, region name, state name"
8,county_fips,Name of county that geotype is in,T,25,NA for geotype RE and CA
9,county_name,FIPS code of county that geotype is in,T,5,2-digit census state code (06) plus 3-digit ce...


**Problem 2b** 

Load the data on Violent Crime in California. 

In [31]:
vc_df = pd.read_csv("ViolentCrime.csv")
vc_df

,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_fips,county_name,...,denominator,rate,ll_95ci,ul_95ci,se,rse,ca_decile,ca_rr,dof_population,version
0,752,"Number of Violent Crimes per 1,000 Population",2000,9,Total,CA,6,California,NaN,NaN,...,33847694.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33873086.0,10/21/15
1,752,"Number of Violent Crimes per 1,000 Population",2000,9,Total,CA,6,California,NaN,NaN,...,33847694.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33873086.0,10/21/15
2,752,"Number of Violent Crimes per 1,000 Population",2000,9,Total,CA,6,California,NaN,NaN,...,33847694.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33873086.0,10/21/15
3,752,"Number of Violent Crimes per 1,000 Population",2000,9,Total,CA,6,California,NaN,NaN,...,33847694.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33873086.0,10/21/15
4,752,"Number of Violent Crimes per 1,000 Population",2000,9,Total,CA,6,California,NaN,NaN,...,33847694.0,6.217499,6.190935,6.244063,0.013553,0.217985,NaN,1.000000,33873086.0,10/21/15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49221,752,"Number of Violent Crimes per 1,000 Population",2013,9,Total,RE,14,Southern California,NaN,NaN,...,18543759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18482091.0,10/21/15
49222,752,"Number of Violent Crimes per 1,000 Population",2013,9,Total,RE,14,Southern California,NaN,NaN,...,18543759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18482091.0,10/21/15
49223,752,"Number of Violent Crimes per 1,000 Population",2013,9,Total,RE,14,Southern California,NaN,NaN,...,18543759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18482091.0,10/21/15
49224,752,"Number of Violent Crimes per 1,000 Population",2013,9,Total,RE,14,Southern California,NaN,NaN,...,18543759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18482091.0,10/21/15


**Problem 2c**

Load the county data. 

In [32]:
cl_df = pd.read_csv('MPO_CountyList.csv',skiprows=1)
cl_df

,County_FIPS,County,MPO Region,MPO_Region_code
0,6001,Alameda,Bay Area,1
1,6013,Contra Costa,Bay Area,1
2,6041,Marin,Bay Area,1
3,6055,Napa,Bay Area,1
4,6075,San Francisco,Bay Area,1
5,6081,San Mateo,Bay Area,1
6,6085,Santa Clara,Bay Area,1
7,6095,Solano,Bay Area,1
8,6097,Sonoma,Bay Area,1
9,6007,Butte,Butte,2


**Problem 2d**

Using the available data identify any trends or correlations present within the data. 

Do you identify any particularly strong correlations? 

In [ ]:
# complete
# complete
# complete
# complete
# complete
# complete
# complete

## Problem 3

Now that you have completed your analysis of Problems 1 and 2, please read the summary questions in the doc: [BoydDatasheet.rtf](./BoydDatasheet.rtf). 

After reading any (or all) of the questions in that doc, consider whether you would like to re-do any of the analysis that you have completed in this notebook. If you do adjust your analysis, please use the cells below associated with Problem 3 so that your initial analysis remains "clean".



*Note* – If you would like to do some additional reading, the full article by Karen Boyd provides the inspiration for the questions: [Datasheets for Datasets Help ML Engineers Notice and
Understand Ethical Issues in Training Data](http://karenboyd.org/Datasheets_Help_CSCW.pdf)

In [ ]:
# complete
# complete
# complete
# complete
# complete
# complete
# complete

In [ ]:
# complete
# complete
# complete
# complete
# complete
# complete
# complete

In [ ]:
# complete
# complete
# complete
# complete
# complete
# complete
# complete

In [ ]:
# complete
# complete
# complete
# complete
# complete
# complete
# complete

## Challenge Problem

Run the first cell below to load data on the efficiency of electric vehicles. 

What EV is the most efficient? (This is not meant to be a difficult question, it is possible to answer this very quickly)

BUT! 

Using what you have learned throughout this notebook, consider again the question of which EV is most efficient? What do you find? 

In [ ]:
# complete
# complete
# complete
# complete
# complete
# complete
# complete